# Code to take data using the scope and plot relevant histograms. Will output energy resolution and associated errors.

In [1]:
import os
import sys
import pyvisa as visa

#sys.path.insert(0, '..\scope-daq')

import MSO4102Bastro as sdaq # This is the scope module that Sean G. wrote. You will need this module (should be on GitHub, https://github.com/ibrewer/scope-daq)
import importlib
#importlib.reload(sdaq)
import time
import h5py # This is the python library that creates files/stores data sets
import numpy as np
import matplotlib.pyplot as plt

### Enter the IP address of the scope as a string. The IP address of the scope should be set by the router (make sure both the scope and the lab laptop are plugged into the router). To check the IP address of the scope, you can go to the Utility menu and check the LAN settings. Sometimes a LAN reset is required.

In [4]:
scope = sdaq.Scope(address="169.254.2.185")

Define output file and data set names

In [3]:
path='../dataOut/101921_amp1/'
if not os.path.exists(path):
    os.makedirs(path)
outName='cobalt57_14h'
dsName='run1'
scope.set_source_channel(0) #Set scope channel to read - same as displayed on scope
traces=100

Boolean run options

In [4]:
savePlots=True 

### This command creates a h5py file, desginated as "f." Documentation for h5py can be found at http://docs.h5py.org/en/stable/. 
### NOTE: Please keep the 'a' flag.  ***Also, make sure you close an open file (use f.close()) before you open a new one.***

In [5]:
g = h5py.File(path+outName+'_scaling.h5py', 'a')

### Create an array that stores the scope scaling dictionary. This will make sure that we have the scope settings for any given run.

### If you want to extract the scope scaling parameters from a file, the 5 settings are stored in the order [x zero, x incr, y zero, y mult, y offset].
### To extract the data, you could say ***data_scale = f['scope_scaling']*** and then extract the values that you want using ***data_scale[1] = x_increment***, for example.

In [6]:
scaling_dict = scope.read_scaling_config()

scaling_info = np.zeros(5)
scaling_info[0] = float(scaling_dict['XZERO'])
scaling_info[1] = float(scaling_dict['XINCR'])
scaling_info[2] = float(scaling_dict['YZERO'])
scaling_info[3] = float(scaling_dict['YMULT'])
scaling_info[4] = float(scaling_dict['YOFF'])

#AMANDA - debug error with same dataset name from hardcoded "scope_scaling"
dset = g.create_dataset(dsName+'_scaling', data=scaling_info)

In [7]:
g.close()

### The get_data function will initialize a run (a full run length is 30000 traces, which takes approx. an hour and a half).

### The naming scheme for data sets is filename_run#, ex. "BC_2019703_0822_45_plateau1_initial_run1."

In [8]:
"""
    This function pulls SiPM pulses from the scope and stores them in arrays.
    
    Parameters
    ----------
    
    no_of_traces: float
        However many traces or SiPM pulses you want to record.
    data_set_name: string
        Name of the data set within the file. The naming scheme for data sets is 
        filename_run#, ex. "BurstCube_PostVibe_Cs137_061419_run1."
        
    Returns
    -------
    
    Data sets. Two data sets should be created, the scope scaling information and 
    The scope data is stored in an array (a h5py data set). h5py data sets are nice 
    because you can splice into them. In this case, the size of the array is determined by the 
    number of traces you want from the scope and number of data points the scope collects for 
    each trace; the size of the array is number of traces by amount of scope points. Each row 
    is a scope trace, so plotting/data analysis is done in a for loop that looks at each row 
    one at a time.
    
    """

def get_data(run_time, data_set_name, no_of_traces = 100, noise_range = (0, 2000), signal_range = (2000,10000)):

    # Determines number of points from each scope trace and creates an empty array
    _, last_trace = scope.read_triggered_event()
    last_trace = np.array([int(s) for s in last_trace.split(',')])
    
    curve_length = len(last_trace)
    #arr = np.zeros((no_of_traces, curve_length))
    arr = [] 
    
    # Scaling dictionary is used to scale the scope traces to account for scope settings
    scaling_dict = scope.read_scaling_config()
    
    peaks = []
    integrals = []
    
    n_bad_comm = 0
    i = 0
    t_start = time.time()
    t_end = t_start + run_time * 60 #run time in minutes
    n_dup = 0
    single = True
    
    time_axis = None
    
    bla = time.strftime('%a, %d %b %Y %H:%M:%S', time.localtime(t_end) )
    
    print(f"Starting {run_time} minute run; will be done at {bla}")
    
    while time.time() < t_end:
        # Code to discount duplicates (when the scope gets stuck on a trigger):
        
        try: 
            _, trace = scope.read_triggered_event()
            trace = np.array([int(s) for s in trace.split(',')])
            if np.sum(trace - last_trace) == 0:
                i -= 1
                n_dup +=1
                print("%d duplicates" %(n_dup))
            else:
                last_trace = trace
                time_scaled, trace_scaled = scope.scale_data(scaling_dict, trace)
                
                if (no_of_traces < 0) or (i < no_of_traces):
                    arr.append(trace_scaled)
                    time_axis = time_scaled
                #if i % 1000 == 0:
                #    print("At {0:d} / {1:d}".format(i, no_of_traces))
                
                noise_sample = np.mean(trace_scaled[noise_range[0]:noise_range[1]])
                trace_scaled -= noise_sample
                peaks.append( np.max(trace_scaled)  )
                integrals.append( np.sum(trace_scaled[signal_range[0]:signal_range[1]] ) )    
                
        
        # Code to override Visa errors:
        except visa.VisaIOError:
            n_bad_comm += 1
            print("Communication timeout... %d" %(n_bad_comm))
            i -= 1 
        
        i += 1       
    
        #if i > 0 and i %1000 == 0 and single:
        #    t_now = time.time()
        #    single = False 
        #    elapsed = time.time() - t_start
        #    rate = float(i)/float(t_now - t_start)
        #    print("{2:s} ; At {0:d}/{1:d}".format(i, no_of_traces, time.strftime('%a, %d %b %Y %H:%M:%S GMT', time.localtime())))
        #    print("\tRate: {0:6.3f} Hz\t Elapsed: {1:6.2f} s\t Estimated total run length: {2:6.2f} s\t Estimated time remaining: {3:6.2f}".format(rate, elapsed, no_of_traces/(rate), no_of_traces/(rate) - elapsed))

        #if i%1000 == 1:
        #    single = True
    
    t_stop = time.time()
    
    run_len = t_stop - t_start
    run_min = run_len / 60
    print(f"Recorded {i} traces in {run_min:0.3f} minutes. Average rate: {i/run_len:.2f} Hz")

    
    dset = f.create_dataset(data_set_name, data=np.array(arr))    
    dset = f.create_dataset(data_set_name+"_t", data=np.array(time_axis))   

    dset = f.create_dataset(data_set_name+"_peaks", data=np.array(peaks))   
    dset = f.create_dataset(data_set_name+"_integral", data=np.array(integrals))   

    return


### Actually take data, double check the data set name. Remember the first function input is the number of traces and the second input is the data set name

In [9]:
f = h5py.File(path+outName+'.h5py', 'a')

In [ ]:
get_data(14*60, dsName, traces) # take data

Starting 840 minute run; will be done at Wed, 20 Oct 2021 06:30:29
1 duplicates


### List the data sets within a file (check to make sure your run is there):

In [ ]:
list(f.keys())

### Data analysis/plotting: assign the data set as the array "plot_array."

In [ ]:
plot_array = f[dsName] # insert desired data set name here
time_axis = np.array(f[dsName+"_t"])
#sanity check
if len(plot_array)!=traces: 
    print("ERROR - SOMETHING AWRY \n dataset length is not what was input - are you looking at the right dataset?")

### Plot 10% of all traces to look at the data and perform a common sense check. Also look for a stretch of data that can be used to determine noise.

In [ ]:
for i in range(int(traces*0.1)):
    plt.plot(time_axis*1e6, plot_array[i])
    
plt.xlabel('time [us]')
plt.ylabel('peak amplitude [V]')

if savePlots:
    plt.savefig(path+outName+'_'+dsName+'_traces.png')
    

### This function will extract the peak from each trace/row of the data set. The noise sample is determined by eye; pick a range of x values where there don't appear to be (many) peaks, ex. from 2000:3500.

In [ ]:
peaks = f[dsName + '_peaks' ]
integ = f[dsName + "_integral"]

### Plot a histogram of the peaks using matplotlib to get a sense of what the data looks like and where the peak is.

In [ ]:
#ultra fine binning for debugging input
plt.hist(peaks, bins=180)
plt.xlabel('peak amplitude [V]')
plt.ylabel('counts')

if savePlots:
    plt.savefig(path+outName+'_'+dsName+'_runtimePeakHist.png')


# Do the same for the trace integral

In [ ]:
#ultra fine binning for debugging input
plt.hist(integ, bins=180)
plt.xlabel('trace integral [V]')
plt.ylabel('counts')

if savePlots:
    plt.savefig(path+outName+'_'+dsName+'_runtimeIntegHist.png')

### If you are done working with a file, ***make sure you close it!*** h5py does not like it when files are left open and you change files and/or kill the kernel.

In [ ]:
f.close()

YOU DID IT!!!